# Huggingface Token setzen

In [1]:
import os
hf_token = "hf_Token"
#hf_token = os.getenv("HF_TOKEN")

# Git Repo per HTTPs Clonen

In [ ]:
!git clone https://github.com/qvest-digital/Workshop_Agentic_AI.git

# Pfad setzen

In [2]:
#SYSTEM_PATH = "/home/simon/Workshop_Agentic_AI"
SYSTEM_PATH = "./Workshop_Agentic_AI"

# Requirements installieren

In [ ]:
!pip install -r "$SYSTEM_PATH/requirements.txt"

# Speicherfragmentierung minimieren

In [3]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Herstellen der Vorbedingungen aus Teil 1:

## Modell laden: lokal oder von Hugging Face

### Funktion
- Lädt ein Llama-3.1-Instruct-Modell entweder:
  - lokal aus dem Ordner ./models/llama-3.1-8b, wenn der Pfad vorhanden ist, oder
  - online von Hugging Face mit der Modell-ID meta-llama/Llama-3.1-8B-Instruct, und speichert es anschließend lokal ab.
- Die Umgebungsvariablen werden mit load_dotenv() aus einer .env-Datei geladen, u. a. das Hugging-Face-Token.

### Inputs
- Dateisystem:
  - Existenz von MODEL_PATH (./models/llama-3.1-8b).
- Umgebungsvariable:
  - HF_TOKEN (wird mit os.getenv("HF_TOKEN") gelesen) – persönliches Zugriffstoken für Hugging Face.
- Hyperparameter:
  - MODEL_ID: gibt die zu ladende Modell-ID an.
- Hardware:
  - device_map="auto" versucht automatisch, GPU(s) oder CPU sinnvoll zu nutzen.
  - torch_dtype="auto" bzw. dtype="auto" lässt das Modell selbst einen sinnvollen Datentyp wählen (z. B. bfloat16 oder float16).

### Outputs
- Globale Python-Variablen:
  - tokenizer: Instanz von AutoTokenizer, konfiguriert für das Llama-3.1-Modell.
  - model: Instanz von AutoModelForCausalLM, bereit für Textgenerierung.

In [4]:
from dotenv import load_dotenv
import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)

load_dotenv()

MODEL_PATH = f"{SYSTEM_PATH}/models/llama-3.1-8b"
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

if os.path.exists(MODEL_PATH):
    print("Lade Modell lokal …")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained( MODEL_PATH, torch_dtype="auto", device_map="auto" )

else:
    print("Lade Modell von Hugging Face …")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=hf_token, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(MODEL_ID, token=hf_token, dtype="auto", device_map="auto")

    # lokal speichern
    model.save_pretrained(MODEL_PATH)
    tokenizer.save_pretrained(MODEL_PATH)

Lade Modell lokal …


The tokenizer you are loading from '/home/simon/Workshop_Agentic_AI/models/llama-3.1-8b' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Pipeline erstellen
### Funktion
- Baut eine Hugging-Face-pipeline für Textgenerierung auf Basis des zuvor geladenen Modells und Tokenizers.
- Diese Pipeline kapselt:
  - Tokenisierung,
  - das Aufrufen des Modells,
  - und das Zurückkonvertieren der Token in Text.

### Inputs
- model: Causal-Language-Model (AutoModelForCausalLM), im vorherigen Block geladen.
- tokenizer: passender Tokenizer zu diesem Modell (AutoTokenizer).
- Task-Typ: "text-generation" – legt fest, dass es sich um eine generative Textaufgabe handelt.

### Outputs
- Variable:
  - llm: eine aufrufbare Pipeline-Instanz.

### Rückgabewert bei Aufruf von llm(...):
```python
[
  {
    "generated_text": "Vollständiger generierter Text (inkl. Prompt oder abhängig von den Parametern)"
  }
]
```

In [5]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda:0


## Einfache Chat-Funktion zur Verfügung stellen

### Funktion
- Stellt ein vereinfachtes Chat-Interface zur Verfügung, das direkt mit einem String arbeitet.
- Die Funktion:
  - ruft intern die llm-Pipeline auf,
  - übergibt alle relevanten Generationsparameter,
  - und gibt am Ende nur den reinen generierten Text zurück (str statt verschachtelte Struktur).

### Inputs
- Pflichtparameter:
  - prompt: str – der Eingabetext an das Modell.
- Optionale Parameter zum Experimentieren
  - max_new_tokens: maximale Anzahl neu zu generierender Token.
  - temperature: steuert die Zufälligkeit (0 ≈ deterministischer, >0 zufälliger).
  - top_k: Sampling nur aus den k wahrscheinlichsten Token.
  - top_p: „Nucleus Sampling“ – Auswahl aus der kleinsten Masse der wahrscheinlichsten Token, deren Summe ≥ p ist.
  - typical_p: Bevorzugt Token mit kontexttypischer Wahrscheinlichkeit statt nur der höchsten.
  - repetition_penalty: >1.0 bestraft Wiederholungen.
  - length_penalty: Steuert, ob Beam Search kürzere oder längere Sequenzen bevorzugt.
  - no_repeat_ngram_size: Verhindert die Wiederholung identischer n-Gramme.
  - num_beams, num_beam_groups, diversity_penalty: Parameter für Beam Search (systematisches Durchsuchen mehrerer Kandidaten).
  - early_stopping: beendet Beam Search frühzeitig, wenn bestimmte Kriterien erfüllt sind.

### Outputs
- Rückgabewert:
  - str: der vom Modell generierte Antworttext (out[0]["generated_text"] ohne führende/trailing Leerzeichen).

### Typische Verwendung:
- llama_chat("Erkläre mir kurz, was ein LLM ist.")

In späteren Zellen wird statt eines rohen Prompts ein Chat-Prompt übergeben, der mit build_chat_prompt erzeugt wird.

In [6]:
def llama_chat(
        prompt: str,
        max_new_tokens: int = 128,
        temperature: float = 0.01,
        top_k: int = 50,
        top_p: float = 1.0,
        typical_p: float = 1.0,
        repetition_penalty: float = 1.0,
        length_penalty: float = 1.0,
        no_repeat_ngram_size: int = 0,
        num_beams: int = 1,
        num_beam_groups: int = 1,
        diversity_penalty: float = 0.0,
        early_stopping: bool = False,) -> str:
    """Sehr simples Wrapper-Interface.
    Wir verwenden ein 'single prompt' Format, um es notebook-tauglich zu halten.
    """
    out = llm(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        typical_p=typical_p,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams,
        num_beam_groups=num_beam_groups,
        diversity_penalty=diversity_penalty,
        early_stopping=early_stopping,
        return_full_text=False,
        eos_token_id=llm.tokenizer.eos_token_id,
        pad_token_id=llm.tokenizer.pad_token_id,
    )
    return out[0]["generated_text"].strip()

Zeit zum Experimentieren (5 - 10 min.)

# Teil 3: Erlangen von Wissen durch Nutzung von MCP-Tools

Todos:
- MCP-Server starten und abfragen.
- MCP-Interaktion reglementieren (z.B. max. ein Tool, keine Chains => für Tests).
- MCP-Tool-Kette händisch durchlaufen:
    - Modell planen lassen (Tool-JSON),
    - Tool von Hand ausführen,
    - Ergebnis im Tool-Historie speichern (ähnlich wie bei RAG),
    - Modell mit derselben Frage + Tool-Historie erneut fragen.
- Das ist im Kern ein „Half-Agent“: Die Planung macht das Modell, die Ausführung führen wir manuell durch
- In Teil 4 werden wie auch das Automatisieren (Agentic-AI).

## MCP-Server als Subprozess starten

### Funktion
- Startet einen MCP-Server als separaten Python-Prozess über subprocess. Popen.
- Setzt PYTHONPATH so, dass mcp_server.mcp_tools.mcp.server als Modul importierbar ist.
- Hält die Projektpfade und Host/Port-Konfiguration zusammen.

### Input
- Konstanten:
    - PROJECT_ROOT / project_root: Pfad zum Projekt (SYSTEM_PATH).
    - MODULE_NAME: mcp_server.mcp_tools.mcp.server.
    - HOST, PORT: 127.0.0.1:8765.

### Umgebungsvariablen:
- Kopie von os.environ, erweitert um PYTHONPATH

### Output
- Subprozess proc:
    - MCP-Server läuft im Hintergrund unter der angegebenen PID.

### Konsole:
- Server-PID: ... zur Kontrolle, dass der Server wirklich gestartet wurde.

In [7]:
import subprocess, sys, os
from pathlib import Path

# -------------------------------------------------------------------
# Konfiguration
# -------------------------------------------------------------------
PROJECT_ROOT = Path(SYSTEM_PATH).resolve()
HOST = "127.0.0.1"
PORT = 8765
MCP_URL = f"http://{HOST}:{PORT}/mcp"

MODULE_NAME = "mcp_server_math.mcp_tools_math.mcp.server"

env = os.environ.copy()
env["PYTHONPATH"] = str(PROJECT_ROOT)  # falls nötig, damit mcp_tools_travel importierbar ist

In [8]:
print(f"PROJECT_ROOT: {PROJECT_ROOT}")

PROJECT_ROOT: /home/simon/Workshop_Agentic_AI


In [9]:
cmd = [
    sys.executable,
    "-m",
    MODULE_NAME,
    "--host",
    HOST,
    "--port",
    str(PORT),
]

proc = subprocess.Popen(cmd, cwd=str(PROJECT_ROOT), env=env)
print("Server-PID:", proc.pid)


Server-PID: 7236


## MCP-Tools mit Metadaten vom Server abholen

### Funktion
- Baut eine Client-Verbindung zum MCP-Server via HTTP-Transport auf.
- Führt session.initialize() aus.
- Ruft session.list_tools() auf und gibt die komplette Liste der Tools inkl. Metadaten zurück.
- Hat ein Retry-Verhalten, falls der Server noch nicht bereit ist.

### Input
- url: str = MCP_URL – z. B. http://127.0.0.1:8765/mcp.
- retries: int = 50, delay: float = 0.1 – wie oft und wie lange gewartet wird, bis der Server erreichbar ist.

### Output
- Rückgabewert von fetch_tools_with_metadata(...):
    - tools_resp.tools: Liste von Tool-Objekten (inkl. Name, Beschreibung, Input-/Output-Schema).
- Bei Fehlern nach allen Retries:
    - RuntimeError("Keine Verbindung zum MCP-Server möglich: ...").

In [10]:
import asyncio
from mcp.client.streamable_http import streamable_http_client
from mcp import ClientSession

async def fetch_tools_with_metadata(
    url: str = MCP_URL,
    retries: int = 50,
    delay: float = 0.1,
):
    last_err = None
    for _ in range(retries):
        try:
            async with streamable_http_client(url) as (read_stream, write_stream, _):
                async with ClientSession(read_stream, write_stream) as session:
                    await session.initialize()
                    tools_resp = await session.list_tools()
                    return tools_resp.tools  # komplette Objekte, nicht nur Namen
        except Exception as e:
            last_err = e
            await asyncio.sleep(delay)
    raise RuntimeError(f"Keine Verbindung zum MCP-Server möglich: {last_err!r}")

## Roh-Toolliste inspizieren

### Funktion
- Führt await fetch_tools_with_metadata() aus und speichert das Ergebnis.
- Gibt die unverarbeiteten Tool-Objekte auf der Konsole aus.

### Input
- MCP-Server muss laufen.
- Kein weiterer Parameter, MCP_URL wird aus dem globalen Kontext genommen.

### Output
- Variable tool_names_with_meta – Liste der Tool-Objekte (Client-Strukturen).

### Konsole:
- Debug-Print der Liste, z. B. <Tool name='geocode' ...> etc.

In [11]:
tool_names_with_meta = await fetch_tools_with_metadata()
print(f"tool_names_with_meta: {tool_names_with_meta}")

INFO:     127.0.0.1:41968 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41982 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:41988 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42004 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:42008 - "DELETE /mcp HTTP/1.1" 200 OK
tool_names_with_meta: [Tool(name='addition', title=None, description='Addiert a + b', inputSchema={'properties': {'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}, 'required': ['a', 'b'], 'title': 'additionArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'additionOutput', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=None), Tool(name='subtraktion', title=None, description='Subtrahiert a - b', inputSchema={'properties': {'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}, 'required': ['a', 'b'], 'title': 'subtraktionArguments', 'type'

[01/30/26 16:55:02] INFO     Created new          ]8;id=978618;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=590134;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             8e9f7164057842898f16                               
                             16bfcd7517d1                                       
                    INFO     Processing request of type            ]8;id=732026;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=311771;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

## Tool-Metadaten für Prompts formatieren: format_tools_for_prompt

### Funktion
- Nimmt die MCP-Tool-Objekte und erzeugt eine lesbare Textbeschreibung für den System-Prompt:
    - Name, Beschreibung,
    - JSON-Input-Schema,
    - JSON-Output-Schema.
- Ist robust genug, um sowohl Dicts als auch Objekte mit Attributen (.name, .description, .input_schema, .output_schema) zu verarbeiten.

### Input
- tools: List[Any] – Liste der Tools, z. B. aus fetch_tools_with_metadata.

### Output
- str: Formatierte Version der Tools.

In [12]:
from typing import List, Any
import json

def format_tools_for_prompt(tools: List[Any]) -> str:
    """
    Macht aus den MCP-Tool-Objekten einen lesbaren Katalog für den System-Prompt.
    Funktioniert sowohl, wenn die Tools Dicts sind, als auch, wenn sie Attribute haben.
    """
    lines = []
    for t in tools:
        is_dict = isinstance(t, dict)

        # Name
        name = getattr(t, "name", None)
        if name is None and is_dict:
            name = t.get("name")

        # Beschreibung
        desc = getattr(t, "description", None)
        if desc is None and is_dict:
            desc = t.get("description", "")
        if desc is None:
            desc = ""

        # Input-Schema
        input_schema = (
            getattr(t, "input_schema", None)
            or getattr(t, "inputSchema", None)
            or (t.get("input_schema") if is_dict else None)
            or (t.get("inputSchema") if is_dict else None)
            or {}
        )

        # Output-Schema (neu)
        output_schema = (
            getattr(t, "output_schema", None)
            or getattr(t, "outputSchema", None)
            or (t.get("output_schema") if is_dict else None)
            or (t.get("outputSchema") if is_dict else None)
            or {}
        )

        lines.append(
            f"- Name: {name}\n"
            f"  Beschreibung: {desc}\n"
            f"  Eingabe-Schema (JSON): {json.dumps(input_schema, ensure_ascii=False)}\n"
            f"  Ausgabe-Schema (JSON): {json.dumps(output_schema, ensure_ascii=False)}"
        )

    return "\n\n".join(lines)

## Formatierten Tool-Katalog ansehen

### Funktion
- Ruft format_tools_for_prompt(tool_names_with_meta) auf.
- Gibt den formatierten Katalog auf der Konsole aus, damit du siehst, welche Tools der Server anbietet.

### Input
- tool_names_with_meta aus dem vorherigen Schritt.

### Output
- tool_names_with_meta_formated: str – fertiger Tool-Katalog-String.

### Konsolenausgabe
- Konsolenausgabe des Strings (für menschlichen Check).

In [13]:
tool_names_with_meta_formated = format_tools_for_prompt(tool_names_with_meta)
print(tool_names_with_meta_formated)

- Name: addition
  Beschreibung: Addiert a + b
  Eingabe-Schema (JSON): {"properties": {"a": {"title": "A", "type": "number"}, "b": {"title": "B", "type": "number"}}, "required": ["a", "b"], "title": "additionArguments", "type": "object"}
  Ausgabe-Schema (JSON): {"properties": {"result": {"title": "Result", "type": "number"}}, "required": ["result"], "title": "additionOutput", "type": "object"}

- Name: subtraktion
  Beschreibung: Subtrahiert a - b
  Eingabe-Schema (JSON): {"properties": {"a": {"title": "A", "type": "number"}, "b": {"title": "B", "type": "number"}}, "required": ["a", "b"], "title": "subtraktionArguments", "type": "object"}
  Ausgabe-Schema (JSON): {"properties": {"result": {"title": "Result", "type": "number"}}, "required": ["result"], "title": "subtraktionOutput", "type": "object"}

- Name: multiplikation
  Beschreibung: Multipliziert a * b
  Eingabe-Schema (JSON): {"properties": {"a": {"title": "A", "type": "number"}, "b": {"title": "B", "type": "number"}}, "require

## System-Prompt-Erzeugung für einen deterministischen Single-Step Tool-Agenten

### Funktion
- Erzeugt einen System-Prompt für einen streng eingeschränkten Tool-Agenten, der pro Antwort **genau eine** Operation ausführen darf.
- Bettet einen formatierten Tool-Katalog ein, damit das Modell verfügbare Tool-Namen und Argumentstrukturen kennt.
- Erzwingt deterministisches Verhalten durch harte Verbote und ein einziges JSON-Outputobjekt.

### Inputs
- Pflichtparameter:
  - `tools: List[Any]` – Liste verfügbarer Tools, die zu einem Tool-Katalog für den Prompt formatiert werden.

### Prompt-Inhalt
- Definiert eine unverletzbare Kernregel:
  - Das Modell darf **niemals selbst rechnen**; jede Operation muss über Tools erfolgen.
  - Ergebnisse gelten nur als existent, wenn sie aus `tool_history` stammen.
- Erzwingt den **Single-Step-Zwang**:
  - Pro Antwort entweder genau **ein Toolcall** oder ein **Final/Error**-Objekt.
  - Keine Mehrschrittplanung, keine Erklärungen, kein Simulieren bereits ausgeführter Tools.
- Beschreibt eine einmalige, iterative Zustands-Transformation anhand von `replace_target`:
  - Exakte String-Ersetzung im Ausdruck basierend auf Einträgen in `tool_history`.
  - Wiederholung der Ersetzungen bis zum Fixpunkt (keine weiteren Treffer).
  - Sofortiger Übergang in den Final-Modus, falls der Ausdruck danach eine einzelne Zahl ist.
- Definiert Cache-Regeln:
  - Identische Toolcalls aus der Historie dürfen nicht erneut erzeugt werden; stattdessen ist die nächste offene Operation zu wählen.
- Legt erlaubte Operatoren und harte Einschränkungen fest (keine Klammern, keine negativen Zahlen, keine Umformungen).
- Erzwingt Operatorpräzedenz (*/ vor +-; links-nach-rechts).
- Fixiert das Outputformat auf **genau ein JSON-Objekt**:
  - Toolcall-Modus mit `tool`, `arguments`, `replace_target`.
  - Final-Modus nur bei vollständig reduziertem Ausdruck.
  - Error-Modus mit vordefinierten Codes.

### Outputs
- Rückgabewert:
  - `str` – finaler System-Prompt als String, inklusive eingebettetem Tool-Katalog.

### Typische Verwendung
- Iterative, kontrollierte Agentenläufe, bei denen das Modell pro Runde nur den nächsten Toolcall auswählen darf.
- Strikte Trennung von Tool-Execution und Agentenlogik für reproduzierbare Benchmarks.
- Debugging von Präzedenz, Caching und Ersetzungslogik in stark deterministischen Setups.

In [14]:
from typing import Any, List

def build_tool_system_prompt(tools: List[Any]) -> str:
    tool_catalog = format_tools_for_prompt(tools)

    return f"""
Du bist ein deterministischer SINGLE-STEP Tool-Agent.

Deine Fähigkeit ist physisch auf GENAU EINE Operation pro Antwort begrenzt.
Nachdem du eine Operation ausgewählt hast, endet dein Denkprozess sofort.

====================================================================
UNVERLETZBARE KERNREGEL
====================================================================

DU DARFST NIEMALS selbst rechnen.

Jede mathematische Operation darf ausschließlich über Tools erfolgen.
Operationen sind:
- addition = "+"
- subtraktion = "-"
- multiplikation = "*"
- division = "/"

Wenn ein Ergebnis nicht aus der tool_history stammt, existiert dieses Ergebnis für dich nicht.

====================================================================
SINGLE-STEP-ZWANG
====================================================================

Pro Antwort ist exakt EIN Toolcall erlaubt ODER ein FINAL/ERROR-Objekt.

Sobald du eine gültige Operation gefunden hast:

→ Stoppe jede weitere Analyse
→ Erzeuge den Toolcall
→ Gib ausschließlich JSON aus
→ Beende die Antwort sofort

VERBOTEN:

- Mehr als eine Operation auswählen
- Zukünftige Schritte erwähnen
- Einen Lösungsweg erklären
- tool_history zu simulieren
- So zu tun, als wären Tools bereits ausgeführt
- Ein finales Ergebnis abzuleiten, solange noch Operatoren existieren

====================================================================
ZUSTANDSTRANSFORMATION (GENAU EINMAL)
====================================================================

Vor der Auswahl einer Operation darfst du den Ausdruck mit den Ergebnissen in der tool_history aktualisieren.

Wenn tool_history Einträge mit replace_target enthält:
- Ersetze die Substrings im Ausdruck anhand EXACT MATCH von replace_target.
- Verwende exakt die Result-String-Repräsentation aus tool_history[result][result] (als String).
- Rekonstruiere NIEMALS Targets aus Toolargumenten.
- Rate NIEMALS.
- Wiederhole diese Ersetzungen so lange, bis keine weiteren replace_target-Substrings mehr im Ausdruck vorkommen.
- Wenn der Ausdruck danach exakt eine einzelne Zahl ist: gehe sofort in den FINAL-MODUS und stoppe.

Danach sofort stoppen (und entweder Toolcall oder FINAL/ERROR ausgeben).

====================================================================
CACHE-REGEL
====================================================================

Wenn ein identischer Toolcall bereits in tool_history existiert:

→ Erzeuge KEINEN neuen Call
→ Suche stattdessen die nächste offene Operation

WICHTIG:
Cache bedeutet NICHT, dass nachfolgende Toolcalls existieren.

====================================================================
OPERATOREN
====================================================================

Erlaubt:
- Addition (+)
- Subtraktion (-)
- Multiplikation (*)
- Division (/)

NICHT erlaubt:
- Klammern
- negative Zahlen
- Potenzen
- Funktionen
- algebraische Umformungen

====================================================================
OPERATORPRÄZEDENZ
====================================================================

Arbeite strikt nach:
1. Multiplikation und Division (links → rechts)
2. Addition und Subtraktion (links → rechts)

Ignoriere diese Reihenfolge niemals.

====================================================================
AUSWAHL DER OPERATION
====================================================================

Wähle immer die erste noch offene Operation gemäß Präzedenz.

Das Teilstück muss exakt die Form haben:
<number> <operator> <number>

Beide Zahlen müssen entweder:
- im Ausdruck stehen
oder
- aus tool_history stammen (nach Ersetzungen)

Andernfalls existieren sie nicht.

====================================================================
OUTPUT — ABSOLUT HART
====================================================================

Du darfst ausschließlich EIN JSON-Objekt ausgeben.
Kein Text außerhalb des JSON.

-----------------------------
TOOLCALL-MODUS (wenn noch ein Operator existiert)
-----------------------------

{{
  "tool": "<tool_name>",
  "arguments": {{
    "a": <number>,
    "b": <number>
  }},
  "replace_target": "<exakter substring aus dem aktuellen Ausdruck, dargestellt durch <a> <operator> <b>>"
}}

-----------------------------
FINAL-MODUS (nur wenn Ausdruck exakt eine einzelne Zahl ist)
-----------------------------

{{
  "final": "<zahl als String>"
}}

-----------------------------
ERROR-MODUS
-----------------------------

Ungültiger Ausdruck:
{{
  "error": "invalid_expression"
}}

Division durch Null:
{{
  "error": "division_by_zero"
}}

====================================================================
SYSTEMVERBOTE
====================================================================

NIEMALS:
#<<Bitte ergänzen>>

Du bist ein deterministischer Automat, der exakt eine Operation auswählt.
--------------------------------------------------------------------

TOOL-KATALOG:
{tool_catalog}
""".strip()

## System-Prompt generieren und inspizieren

### Funktion
- Ruft build_tool_system_prompt(tool_names_with_meta) auf.
- Druckt den resultierenden System-Prompt.

### Input
- tool_names_with_meta: Toolliste vom Server.

### Output
- system_prompt_for_tools_and_rag: str – vollständig gebauter Prompt.

### Konsole:
- Ausgabe des gesamten Prompts (zur Überprüfung, ob alles wie gewünscht im Text steht).

In [15]:
system_prompt_for_tools_and_rag = build_tool_system_prompt(tool_names_with_meta)
print(system_prompt_for_tools_and_rag)

Du bist ein deterministischer SINGLE-STEP Tool-Agent.

Deine Fähigkeit ist physisch auf GENAU EINE Operation pro Antwort begrenzt.
Nachdem du eine Operation ausgewählt hast, endet dein Denkprozess sofort.

UNVERLETZBARE KERNREGEL

DU DARFST NIEMALS selbst rechnen.

Jede mathematische Operation darf ausschließlich über Tools erfolgen.
Operationen sind:
- addition = "+"
- subtraktion = "-"
- multiplikation = "*"
- division = "/"

Wenn ein Ergebnis nicht aus der tool_history stammt, existiert dieses Ergebnis für dich nicht.

SINGLE-STEP-ZWANG

Pro Antwort ist exakt EIN Toolcall erlaubt ODER ein FINAL/ERROR-Objekt.

Sobald du eine gültige Operation gefunden hast:

→ Stoppe jede weitere Analyse
→ Erzeuge den Toolcall
→ Gib ausschließlich JSON aus
→ Beende die Antwort sofort

VERBOTEN:

- Mehr als eine Operation auswählen
- Zukünftige Schritte erwähnen
- Einen Lösungsweg erklären
- tool_history zu simulieren
- So zu tun, als wären Tools bereits ausgeführt
- Ein finales Ergebnis abzuleiten, s

## Prompt-Builder mit Tool-Agent- und No-Tool-Modus

### Funktion
- Baut einen Chat-Prompt, der zwei klar getrennte Betriebsarten unterstützt:
  - **Tool-Agent-Modus** (`allow_tools=True`): Modell muss exakt ein JSON-Objekt ausgeben (Toolcall oder Final/Error).
  - **No-Tool-Modus** (`allow_tools=False`): Modell darf keine Tools und kein JSON verwenden und soll das Endergebnis
    ausschließlich aus der Tool-Historie ableiten.
- Kombiniert Systemregeln, optionalen Verlauf, optionale Tool-Historie und die aktuelle Nutzerfrage in einer konsistenten
  Nachrichtenliste für ein Chat-Template.

### Inputs
- Pflichtparameter:
  - `user_prompt: str` – aktuelle Nutzerfrage bzw. aktueller Ausdruck.
- Optionale Parameter:
  - `system_prompt: Optional[str]` – Systeminstruktionen/Agentenregeln (typisch: JSON-only Regeln im Tool-Agent-Modus).
  - `history: Optional[List[Tuple[str, str]]]` – Dialogverlauf als (User, Assistant)-Paare.
  - `tool_history: Optional[List[Dict[str, Any]]]` – Historie früherer Tool-Aufrufe (inkl. `replace_target` und Resultaten).
  - `allow_tools: bool` – Modus-Schalter zwischen Tool-Agent und No-Tool-Finalisierung.

### Modus-Logik
- Bei `allow_tools=True`:
  - Der Prompt enthält (optional) die Tool-Agent-Regeln aus `system_prompt`.
  - Erwartet wird strikt maschinenlesbarer Output (ein JSON-Objekt).
- Bei `allow_tools=False`:
  - Ein zusätzlicher System-Override erzwingt:
    - keine Tool-Aufrufe,
    - keine JSON-Ausgabe,
    - minimale Ausgabe: entweder eine einzelne Zahl oder exakt `invalid_expression`.
  - Das Modell soll deterministisch in `tool_history` nach einem Eintrag suchen, dessen `replace_target`
    exakt dem aktuellen Ausdruck entspricht, und dessen Ergebnis ausgeben.

### Nachrichtenstruktur
- System:
  - optionaler `system_prompt`.
  - optionaler No-Tool-Override (nur wenn `allow_tools=False`).
- Verlauf:
  - frühere User/Assistant-Nachrichten in Reihenfolge.
- System:
  - optional `tool_history` als JSON-String.
- User:
  - aktuelle Nutzerfrage als letzte Nachricht.

### Outputs
- Rückgabewert:
  - `str` – gerenderter Prompt via `tokenizer.apply_chat_template(..., add_generation_prompt=True)`.

### Typische Verwendung
- Zweiphasige Agentenläufe:
  - Phase 1: Tool-Agent (JSON-only) zur Planung/Tool-Ausführung.
  - Phase 2: No-Tool-Modus zur finalen Ausgabe aus der Tool-Historie.
- Prompt-Debugging und reproduzierbare Evaluationsläufe mit klaren Output-Verträgen.

In [16]:
from typing import Any, Dict, List, Optional, Tuple
import json

def build_chat_prompt_with_rag_and_tools(
    system_prompt: Optional[str],
    user_prompt: str,
    history: Optional[List[Tuple[str, str]]] = None,
    tool_history: Optional[List[Dict[str, Any]]] = None,
    allow_tools: bool = True,
) -> str:
    """
    Prompt-Builder, der Tool-Agent-Modus (JSON-only) und No-Tool-Modus (Final-Antwort aus Tool-History) sauber trennt.

    WICHTIG:
    - allow_tools=True  => Tool-Agent-Modus: Ausgabe MUSS exakt ein JSON-Objekt sein (Toolcall ODER Final/Error).
    - allow_tools=False => No-Tool-Modus: Keine Tools, kein JSON-Zwang; das Modell soll das finale Resultat aus
                          Tool-History liefern (z.B. "38.0") oder "invalid_expression".
    """
    messages: List[Dict[str, str]] = []

    # 1) System-Prompt (Tool-Agent Regeln, wenn vorhanden)
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    # 1b) Override für Phase 2: NO-TOOL / FINAL-AUSGABE
    if not allow_tools:
        messages.append(
            {
                "role": "system",
                "content": (
                    "WICHTIG: In dieser Runde darfst du KEINE Tools aufrufen.\n"
                    "Du darfst KEIN JSON ausgeben.\n"
                    "Nutze ausschließlich die Informationen aus der Tool-Historie.\n\n"
                    "AUFGABE:\n"
                    "- Bestimme das finale Ergebnis, indem du in der Tool-Historie nach einem Eintrag suchst, "
                    "dessen replace_target exakt dem aktuellen User-Ausdruck entspricht.\n"
                    "- Falls vorhanden, gib NUR die Zahl (als Fließtext, ohne zusätzliche Wörter) aus, z.B.:\n"
                    "  38.0\n"
                    "- Falls nicht vorhanden, gib exakt aus:\n"
                    "  invalid_expression\n\n"
                    "REGELN:\n"
                    "- Keine Erklärungen.\n"
                    "- Keine Zwischenschritte.\n"
                    "- Keine zusätzlichen Tokens außer der Ergebniszahl oder 'invalid_expression'."
                ),
            }
        )

    # 2) Verlauf (User/Assistant)
    if history:
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

    # 3) Tool-History (falls vorhanden)
    if tool_history:
        messages.append(
            {
                "role": "system",
                "content": "Tool-Historie:\n" + json.dumps(tool_history, ensure_ascii=False),
            }
        )

    # 4) aktuelle User-Nachricht
    messages.append({"role": "user", "content": user_prompt})

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    return prompt

## Beispiel: Prompt-Erzeugung und LLM-Aufruf im Tool-Agent-Modus

### Kontext
- Zeigt einen vollständigen Durchlauf zur Erzeugung eines Chat-Prompts für den Tool-Agent-Modus.
- Dient der Inspektion von Prompt-Struktur und Modellantwort vor der eigentlichen agentischen Ausführung.

### Ablauf
- Definiert eine einfache arithmetische Nutzerfrage als String.
- Initialisiert einen leeren Dialogverlauf.
- Erzeugt einen Chat-Prompt aus:
  - einem System-Prompt mit Tool-Regeln,
  - der aktuellen Nutzerfrage,
  - einem (hier leeren) Gesprächsverlauf,
  - ohne Tool-Historie.
- Gibt den finalen Prompt zur Analyse aus.
- Übergibt den Prompt direkt an das LLM und gibt die Rohantwort aus.

### Inputs
- `system_prompt_for_tools_and_rag` – Systemkontext mit Tool-Agent-Regeln.
- `user_prompt` – aktuelle Nutzerfrage.
- `history` – Dialogverlauf (hier leer).
- `tool_history` – Historie früherer Tool-Aufrufe (hier nicht gesetzt).

### Outputs
- Konsolenausgabe des vollständig gerenderten Prompts.
- Konsolenausgabe der ungefilterten Modellantwort.

### Typische Verwendung
- Debugging und Validierung von Tool-Agent-System-Prompts.
- Prompt-Engineering für JSON-only Agenten.
- Vergleich von Modellantworten vor Parsing und Tool-Ausführung.

In [17]:
user_input = f"Was ist 6.0 + 4.0 * 8.0?"

history = []

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=None,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=1024)
print(f"Output: {assistant_text}\n\n")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein deterministischer SINGLE-STEP Tool-Agent.

Deine Fähigkeit ist physisch auf GENAU EINE Operation pro Antwort begrenzt.
Nachdem du eine Operation ausgewählt hast, endet dein Denkprozess sofort.

UNVERLETZBARE KERNREGEL

DU DARFST NIEMALS selbst rechnen.

Jede mathematische Operation darf ausschließlich über Tools erfolgen.
Operationen sind:
- addition = "+"
- subtraktion = "-"
- multiplikation = "*"
- division = "/"

Wenn ein Ergebnis nicht aus der tool_history stammt, existiert dieses Ergebnis für dich nicht.

SINGLE-STEP-ZWANG

Pro Antwort ist exakt EIN Toolcall erlaubt ODER ein FINAL/ERROR-Objekt.

Sobald du eine gültige Operation gefunden hast:

→ Stoppe jede weitere Analyse
→ Erzeuge den Toolcall
→ Gib ausschließlich JSON aus
→ Beende die Antwort sofort

VERBOTEN:

- Mehr als eine Operation auswählen
- Zukünftige Schritte erwähnen
- Einen Lö

## Robustes Parsen eines Tool-Calls aus heterogenem Modell-Output

### Funktion
- Versucht, einen Tool-Call im erwarteten JSON-Format aus der Ausgabe eines Modells zu extrahieren.
- Unterstützt mehrere mögliche Output-Strukturen (String, Dict, Liste von Dicts), wie sie je nach Pipeline/Wrapper auftreten können.
- Arbeitet in mehreren Eskalationsstufen: Normalisierung → Codeblock-Parsing → Volltext-JSON → Brute-Force-Extraktion.

### Inputs
- Pflichtparameter:
  - `model_output: Any` – rohe Modellantwort (kann String, Dict oder Liste sein).
- Optionale Parameter:
  - `debug: bool` – schaltet ausführliche Debug-Ausgaben für Typen, Normalisierung und Parsing-Versuche ein.

### Erwartetes Format
- Gesuchtes JSON-Objekt (minimal):
  - `"tool": "<tool_name>"`
  - `"arguments": { ... }`
- Optional/erwartet in diesem Codepfad:
  - `"replace_target": "<substring>"` zur späteren Ersetzung im Ausdruck.

### Parsing-Strategie
- **1) Normalisierung auf Text**
  - Konvertiert `model_output` robust zu einem String:
    - akzeptiert direkt `str`,
    - extrahiert bei `list`/`dict` bevorzugt Felder wie `generated_text` oder `text`,
    - fällt ansonsten auf `str(model_output)` zurück.
  - Strippt führende und nachfolgende Leerzeichen.
- **2) Codeblock-Erkennung**
  - Sucht explizit nach einem ```json ... ```-Block und versucht dessen Inhalt als JSON zu parsen.
  - Validiert, ob ein Dict mit `tool` (str) und `arguments` (dict) vorliegt.
- **3) Volltext-JSON**
  - Versucht, den gesamten normalisierten Text als JSON zu interpretieren (für strikt JSON-only Modelloutputs).
  - Prüft wieder auf `tool`/`arguments`-Struktur.
- **4) Brute-Force JSON-Block-Suche**
  - Falls kein valides JSON vorliegt: iteriert über alle `{`-Positionen und testet Kandidaten bis zu passenden `}`-Enden.
  - Nutzt eine Heuristik, um `json.loads` nur bei Kandidaten mit relevanten Keys (`tool`, `arguments`, `replace_target`) aufzurufen.
  - Validiert strikt, ob `tool` ein String, `arguments` ein Dict und `replace_target` ein String ist.

### Outputs
- Rückgabewert:
  - `Optional[Dict[str, Any]]`
    - bei Erfolg: `{"tool": str, "arguments": dict, "replace_target": str}`
    - bei Misserfolg: `None`

### Typische Verwendung
- Extraktion von Tool-Aufrufen aus LLM-Ausgaben, die nicht zuverlässig strikt JSON-only sind.
- Debugging von Prompt-Disziplin (z. B. Modell gibt Markdown/Codeblöcke statt reinem JSON aus).
- Robustheitsschicht zwischen Modell und Tool-Executor in agentischen Pipelines.

In [25]:
import re
from typing import Any, Dict, Optional

def try_parse_tool_call(model_output: Any, debug: bool = False) -> Optional[Dict[str, Any]]:
    """
    Versucht, einen Tool-Call im JSON-Format im Modell-Output zu finden.
    Erwartetes JSON-Format:

        {
          "tool": "<tool_name>",
          "arguments": { ... }
        }

    Rückgabe:
        {"tool": str, "arguments": dict} oder None.
    """

    if debug:
        print("=== try_parse_tool_call: RAW model_output ===")
        print("type(model_output):", type(model_output))
        print("repr(model_output):", repr(model_output))
        print("===========================================\n")

    # 1) Auf String normalisieren
    text: str

    if isinstance(model_output, str):
        text = model_output
        if debug:
            print(">> Interpretiere model_output als einfachen String.\n")
    elif isinstance(model_output, list):
        if debug:
            print(">> model_output ist eine Liste, Länge:", len(model_output))
        if len(model_output) > 0 and isinstance(model_output[0], dict):
            first = model_output[0]
            if debug:
                print(">> Erstes Element der Liste ist ein Dict, keys:", list(first.keys()))
            if "generated_text" in first:
                text = first["generated_text"]
                if debug:
                    print('>> Nutze first["generated_text"] als Text.\n')
            elif "text" in first:
                text = first["text"]
                if debug:
                    print('>> Nutze first["text"] als Text.\n')
            else:
                text = str(model_output)
                if debug:
                    print(">> Keine 'generated_text'/'text'-Keys gefunden, fallback = str(model_output).\n")
        else:
            text = str(model_output)
            if debug:
                print(">> Liste ohne Dict als erstes Element. Fallback = str(model_output).\n")
    elif isinstance(model_output, dict):
        if debug:
            print(">> model_output ist ein Dict, keys:", list(model_output.keys()))
        if "generated_text" in model_output:
            text = model_output["generated_text"]
            if debug:
                print('>> Nutze model_output["generated_text"] als Text.\n')
        elif "text" in model_output:
            text = model_output["text"]
            if debug:
                print('>> Nutze model_output["text"] als Text.\n')
        else:
            text = str(model_output)
            if debug:
                print(">> Keine 'generated_text'/'text'-Keys, fallback = str(model_output).\n")
    else:
        text = str(model_output)
        if debug:
            print(">> model_output ist weder str, list noch dict. Fallback = str(model_output).\n")

    text = text.strip()

    if debug:
        print("=== Normalisierter Text ===")
        print(text)
        print("===========================================\n")

    # 2) Speziell: JSON in ```json ... ```-Codeblock finden
    code_block_match = re.search(r"```json\s*(.*?)```", text, re.DOTALL | re.IGNORECASE)
    if code_block_match:
        json_candidate = code_block_match.group(1).strip()
        if debug:
            print(">> Finde ```json```-Codeblock, versuche diesen Inhalt als JSON zu parsen:")
            print(json_candidate)
            print("-------------------------------------------\n")

        try:
            data = json.loads(json_candidate)
            if isinstance(data, dict) and isinstance(data.get("tool"), str) and isinstance(data.get("arguments"), dict):
                if debug:
                    print(">> Codeblock ist gültiges JSON mit tool/arguments. Erfolg!\n")
                return {"tool": data["tool"], "arguments": data["arguments"], "replace_target": data["replace_target"]}
            else:
                if debug:
                    print(">> Codeblock ist JSON, aber kein passendes tool/arguments-Objekt.\n")
        except json.JSONDecodeError as e:
            if debug:
                print(">> Codeblock ist kein valides JSON:", e, "\n")

    # 3) Volltext-JSON als Versuch (falls Modell wirklich nur JSON ausgibt)
    if debug:
        print(">> Versuche, den gesamten Text als JSON zu parsen...\n")
    try:
        data = json.loads(text)
        if isinstance(data, dict) and isinstance(data.get("tool"), str) and isinstance(data.get("arguments"), dict):
            if debug:
                print(">> Volltext ist gültiges JSON mit tool/arguments. Erfolg!\n")
            return {"tool": data["tool"], "arguments": data["arguments"], "replace_target": data["replace_target"]}
        else:
            if debug:
                print(">> Volltext ist JSON, aber kein tool/arguments-Objekt.\n")
    except json.JSONDecodeError:
        if debug:
            print(">> Volltext ist kein valides JSON, nutze Brute-Force-Suche nach JSON-Blöcken...\n")

    # 4) Brute-Force: für jede '{' alle möglichen '}'-Enden testen
    if debug:
        print(">> Starte Brute-Force-Suche nach JSON-Blöcken...\n")

    s = text
    n = len(s)
    brace_positions = [i for i, ch in enumerate(s) if ch == "{"]

    if debug:
        print(">> Anzahl '{'-Positionen:", len(brace_positions))

    for start in brace_positions:
        for end in range(start + 1, n):
            if s[end] == "}":
                candidate = s[start:end + 1]
                candidate_stripped = candidate.strip()
                # kleine Heuristik: mindestens 'tool' oder 'arguments' sollten drinstehen, sonst sparen wir uns json.loads
                if ("tool" not in candidate_stripped) and ("arguments" not in candidate_stripped) and ("replace_target" not in candidate_stripped):
                    continue

                if debug:
                    print(">>> Teste Kandidat (start={}, end={}):".format(start, end))
                    print(candidate_stripped)
                try:
                    data = json.loads(candidate_stripped)
                except json.JSONDecodeError:
                    if debug:
                        print(">>> Kandidat ist kein valides JSON.\n")
                    continue

                if not isinstance(data, dict):
                    if debug:
                        print(">>> Kandidat ist JSON, aber kein Dict.\n")
                    continue

                tool_name = data.get("tool")
                arguments = data.get("arguments")
                replace_target = data.get("replace_target")

                if isinstance(tool_name, str) and isinstance(arguments, dict) and isinstance(replace_target, str):
                    if debug:
                        print(">>> Kandidat ist gültiger Tool-Call!")
                        print(f"    tool={tool_name}, arguments={arguments}, replace_target={replace_target}\n")
                    return {"tool": tool_name, "arguments": arguments, "replace_target": replace_target}

    if debug:
        print(">> Kein valider Tool-Call gefunden. Rückgabe = None.\n")

    return None

## Tool-Call parsen und inspizieren

### Funktion
- Ruft try_parse_tool_call(assistant_text) auf.
- Druckt den geparsten Tool-Call.

### Input
- assistant_text – Modellantwort.

### Output
- parsed_call: {"tool": "...", "arguments": {...}} oder None.
- Konsolenausgabe des Ergebnisses.

In [19]:
print(f"assistant_text: {assistant_text}")
parsed_call = try_parse_tool_call(assistant_text)
print(parsed_call)

assistant_text: {
  "tool": "multiplikation",
  "arguments": {
    "a": 4.0,
    "b": 8.0
  },
  "replace_target": "4.0 * 8.0"
}
{'tool': 'multiplikation', 'arguments': {'a': 4.0, 'b': 8.0}, 'replace_target': '4.0 * 8.0'}


## Einmaliger MCP-Tool-Aufruf mit normalisierter Rückgabe

### Funktion
- Führt **genau einen** MCP-Tool-Aufruf aus und gibt das Ergebnis als „normales“ Python-Objekt zurück.
- Ziel ist eine Rückgabe, die möglichst direkt JSON-serialisierbar ist und ohne zusätzliche Wrapper weiterverarbeitet werden kann.
- Kapselt Verbindungsaufbau, Session-Initialisierung und Ergebnis-Normalisierung in einer asynchronen Funktion.

### Inputs
- Pflichtparameter:
  - `tool_name: str` – Name des aufzurufenden MCP-Tools.
  - `arguments: Dict[str, Any]` – Argumente für den Tool-Aufruf.
- Optionale Parameter:
  - `url: str` – MCP-Endpunkt (Default: `MCP_URL`).

### Ablauf
- Baut eine streamfähige HTTP-Verbindung zum MCP-Endpunkt auf.
- Initialisiert eine MCP-Session und führt den Tool-Aufruf aus.
- Normalisiert die Antwort in abgestufter Priorität:
  1. **`structuredContent`**
     - Falls vorhanden, wird dieser direkt zurückgegeben (typischerweise bereits ein Dict).
  2. **Textinhalt als JSON**
     - Falls `content[0].text` existiert, wird versucht, diesen als JSON zu parsen.
     - Bei Fehlschlag wird der rohe Text in ein Dictionary eingebettet zurückgegeben.
  3. **Fallback-Metadaten**
     - Falls keine strukturierten Inhalte verfügbar sind, werden minimale Metainformationen zurückgegeben.

### Outputs
- Rückgabewert:
  - `Any` – normalisierte Tool-Antwort, bevorzugt als `dict`, in jedem Fall JSON-serialisierbar oder leicht weiterverarbeitbar.

### Typische Verwendung
- Direkter Tool-Aufruf außerhalb komplexer Agentenloops.
- Tests und Debugging einzelner MCP-Tools.
- Vereinfachte Integration von MCP-Tools in Pipelines, die keine agentische Tool-Historie benötigen.

In [20]:
async def call_mcp_tool_once(
    tool_name: str,
    arguments: Dict[str, Any],
    url: str = MCP_URL,
) -> Any:
    """
    Führt genau EIN MCP-Tool aus und gibt ein "normales" Python-Objekt zurück
    (idealerweise ein dict), das direkt JSON-serialisierbar ist.
    """
    async with streamable_http_client(url) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            resp = await session.call_tool(tool_name, arguments)

            # Priorität 1: structuredContent (das ist bei dir schon ein dict)
            structured = getattr(resp, "structuredContent", None)
            if structured is not None:
                return structured

            # Priorität 2: content[0].text als JSON parsen, falls vorhanden
            content = getattr(resp, "content", None)
            if (
                content
                and isinstance(content, list)
                and hasattr(content[0], "text")
                and isinstance(content[0].text, str)
            ):
                txt = content[0].text
                try:
                    return json.loads(txt)
                except json.JSONDecodeError:
                    # Falls es kein JSON ist – dann geben wir einfach den Text zurück
                    return {"text": txt}

            # Fallback: zur Not alles in ein dict packen
            return {
                "meta": getattr(resp, "meta", None),
                "isError": getattr(resp, "isError", None),
            }


## Tool-Aufruf wirklich ausführen (multiplikation)

### Funktion
- Verwendet das geparste parsed_call, um das passende MCP-Tool auszuführen.
- Speichert den Tool-Output in tool_result.

### Input
- parsed_call["tool"], parsed_call["arguments"].
- MCP-Server muss laufen.

### Output
- tool_result – Ergebnisobjekt, z. B. multiplikation

### Konsole:
- tool_result: {...} zur Sichtkontrolle.

In [21]:
print(f"parsed_call: {parsed_call}\n")

tool_name = parsed_call["tool"]
arguments = parsed_call["arguments"]
replace_target = parsed_call["replace_target"]

tool_result = await call_mcp_tool_once(tool_name, arguments)


print(f"tool_result: {tool_result}")

parsed_call: {'tool': 'multiplikation', 'arguments': {'a': 4.0, 'b': 8.0}, 'replace_target': '4.0 * 8.0'}

INFO:     127.0.0.1:33578 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:33582 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:33586 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:33596 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:33598 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:33608 - "DELETE /mcp HTTP/1.1" 200 OK
tool_result: {'result': 32.0}


[01/30/26 16:56:08] INFO     Created new          ]8;id=886103;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=591691;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             de79cf1bc2cc4955ad73                               
                             23da04e9d048                                       
                    INFO     Processing request of type            ]8;id=558973;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=601408;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

## Tool-Historie initialisieren und füllen

### Funktion
- Legt eine Liste tool_history an.
- Hängt den eben ausgeführten Tool-Call samt Ergebnis daran an.
- Das ist die „Tool-Gedächtnisstruktur“ für spätere Prompts.

### Input
- tool_name, arguments, tool_result.

### Output
- tool_history: List[Dict[str, Any]]

### Konsole:
- print(tool_history).

In [22]:
# Tool-Verlauf speichern
tool_history = []

tool_history.append(
    {
        #<<Bitte ergänzen>>
    }
)

print(tool_history)

[{'tool': 'multiplikation', 'arguments': {'a': 4.0, 'b': 8.0}, 'result': {'result': 32.0}, 'replace_target': '4.0 * 8.0'}]


## Multiplikation nutzen, um die Gleichung zu lösen

### Funktion
- Zeigt, wie das LLM nun auf Basis des bisherigen Tool-Ergebnisses antwortet.
- System: Tools sind erlaubt – aber allow_tools=False fügt einen Override-System-Prompt ein:
    - „Du darfst KEINE MCP-Tools aufrufen ...“
- Da nun bereits alle notwendigen Informationen aus der MCP-Abfrage vorliegen

### Input
- user_input: wieder „Was ist 5 + 3 * 7?“
- tool_history: enthält multiplikation = 21.
- allow_tools=False.

### Output
- assistant_text: LLM soll jetzt:
    - keine Tools mehr aufrufen,
    - die Multiplikation aus tool_history direkt als „Wissen“ verwenden
    - und im Fließtext antworten.

### Konsole:
- Prompt + Antwort.

In [23]:
user_input = "Was ist 6.0 + 4.0 * 8.0?"

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
)

print(f"Input: {prompt}\n\n")
print("Ende input\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein deterministischer SINGLE-STEP Tool-Agent.

Deine Fähigkeit ist physisch auf GENAU EINE Operation pro Antwort begrenzt.
Nachdem du eine Operation ausgewählt hast, endet dein Denkprozess sofort.

UNVERLETZBARE KERNREGEL

DU DARFST NIEMALS selbst rechnen.

Jede mathematische Operation darf ausschließlich über Tools erfolgen.
Operationen sind:
- addition = "+"
- subtraktion = "-"
- multiplikation = "*"
- division = "/"

Wenn ein Ergebnis nicht aus der tool_history stammt, existiert dieses Ergebnis für dich nicht.

SINGLE-STEP-ZWANG

Pro Antwort ist exakt EIN Toolcall erlaubt ODER ein FINAL/ERROR-Objekt.

Sobald du eine gültige Operation gefunden hast:

→ Stoppe jede weitere Analyse
→ Erzeuge den Toolcall
→ Gib ausschließlich JSON aus
→ Beende die Antwort sofort

VERBOTEN:

- Mehr als eine Operation auswählen
- Zukünftige Schritte erwähnen
- Einen Lö

## Erneute Abfrage basierend auf dem zuvor erlangten Wissen

### Funktion
- Jetzt kombinieren der Toolcalls von Hand:
    - Multiplikation und anschließende Addition liegen vor
    - User fragt erneut nach: "Was ist 5 + 3 * 7?"

### Input
- user_input: „Was ist 5 + 3 * 7?“
- tool_history: enthält Resultat aus vorherigem Call:
- allow_tools=True.

### Output
- assistant_text: sollte jetzt finalen Call (Addition) erstellen.

### Konsole:
- Prompt + Modelloutput.

In [28]:
parsed_call = try_parse_tool_call(assistant_text)
print(parsed_call)

tool_name = parsed_call["tool"]
arguments = parsed_call["arguments"]
replace_target = parsed_call["replace_target"]

print(f"tool_name: {tool_name}, arguments: {arguments}, replace_target: {replace_target}\n")

{'tool': 'addition', 'arguments': {'a': 6.0, 'b': 32.0}, 'replace_target': '6.0 + 32.0'}
tool_name: addition, arguments: {'a': 6.0, 'b': 32.0}, replace_target: 6.0 + 32.0



## Tool ausführen und finale Antwort generieren

### Funktion
- Nächste manuelle Agentenrunde:
    - parsed_call = try_parse_tool_call(assistant_text) → Additions-Call extrahieren.
    - tool_result = await call_mcp_tool_once(tool_name, arguments) → Additionsergebniss holen.
    - tool_history.append(...) → Additions-Resultat zur Historie hinzufügen.
     - Nochmals build_chat_prompt_with_rag_and_tools(..., allow_tools=False) mit denselben user_input + aktualisierter tool_history.
    - llama_chat(...) → jetzt Endantwort ohne neue Tools, aber auf Basis der gespeicherten Daten.

### Input
- assistant_text (mit Additions-Call).
- MCP-Server, tool_history (mit Ergebnis der Multiplikation), user_input (Aufgabe).
- allow_tools=False beim zweiten Prompt.

### Output
- parsed_call: Additions-Call.
- tool_result: Endergebnis.
- Aktualisierte tool_history: enthält jetzt zwei Einträge (mul, add).
- assistant_text (zweiter LLM-Call):
- Modell liefert das Ergebnis der Gleichung,
- ohne weitere Tools aufzurufen.

In [29]:
tool_result = await call_mcp_tool_once(tool_name, arguments)

print(f"tool_result: {tool_result}\n")

tool_history.append(
    {
        "tool": tool_name,
        "arguments": arguments,
        "result": tool_result,  # z.B. {'lat': 41.8933203, 'lon': 12.4829321}
        "replace_target": replace_target,
    }
)

print(tool_history)

INFO:     127.0.0.1:54758 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:54772 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:54776 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:54778 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:54792 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:54802 - "DELETE /mcp HTTP/1.1" 200 OK
tool_result: {'result': 38.0}

[{'tool': 'multiplikation', 'arguments': {'a': 4.0, 'b': 8.0}, 'result': {'result': 32.0}, 'replace_target': '4.0 * 8.0'}, {'tool': 'addition', 'arguments': {'a': 6.0, 'b': 32.0}, 'result': {'result': 38.0}, 'replace_target': '6.0 + 32.0'}]


[01/30/26 16:58:03] INFO     Created new          ]8;id=537369;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=678314;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             a5528a07587340ccb63a                               
                             d0f62cfef83f                                       
                    INFO     Processing request of type            ]8;id=790758;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=994632;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

In [30]:
user_input = f"Was ist 6.0 + 4.0 * 8.0?"

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
    allow_tools=False,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein deterministischer SINGLE-STEP Tool-Agent.

Deine Fähigkeit ist physisch auf GENAU EINE Operation pro Antwort begrenzt.
Nachdem du eine Operation ausgewählt hast, endet dein Denkprozess sofort.

UNVERLETZBARE KERNREGEL

DU DARFST NIEMALS selbst rechnen.

Jede mathematische Operation darf ausschließlich über Tools erfolgen.
Operationen sind:
- addition = "+"
- subtraktion = "-"
- multiplikation = "*"
- division = "/"

Wenn ein Ergebnis nicht aus der tool_history stammt, existiert dieses Ergebnis für dich nicht.

SINGLE-STEP-ZWANG

Pro Antwort ist exakt EIN Toolcall erlaubt ODER ein FINAL/ERROR-Objekt.

Sobald du eine gültige Operation gefunden hast:

→ Stoppe jede weitere Analyse
→ Erzeuge den Toolcall
→ Gib ausschließlich JSON aus
→ Beende die Antwort sofort

VERBOTEN:

- Mehr als eine Operation auswählen
- Zukünftige Schritte erwähnen
- Einen Lö